In [ ]:
!pip install mord
import sys
from IPython.core.interactiveshell import InteractiveShell
import warnings
import numpy as np # Array 
%matplotlib inline
import pandas as pd #big data framework
import matplotlib # data visualization
import matplotlib.pyplot as plt
import seaborn as sns #visualization
from sklearn.manifold import TSNE #predictive modeling / Machine Learning
import mord  # python3 -m pip install --user
from sklearn import datasets, neighbors, linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/home/aswin/Documents/Friday/dataset_autoinsurance.csv')

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df = df.drop(['sf_class_tpl', 'sf_class_fc'], axis=1)

In [ ]:
def convert_number_of_payment_faults(s):
   if s == '':
       return 0
   return min(int(s), 9)

In [ ]:
print(df.number_of_payment_faults.value_counts().sort_index())

In [ ]:
pd.get_dummies(df.tariff_type),
pd.get_dummies(df.type_of_insurance),
pd.get_dummies(df.comprehensive_product),
pd.get_dummies(df.payment_interval),
pd.get_dummies(df.insured_parties),
pd.get_dummies(df.profession_group)

In [ ]:
X = df[[col for col in df.columns if col != 'number_of_payment_faults']]

In [ ]:
y = df['number_of_payment_faults']

In [ ]:
standard_scaler = StandardScaler()
df = standard_scaler.fit_transform(X)
tsne = TSNE(n_components=2, random_state=0)
x_test_2d = tsne.fit_transform(df)
#Build the scatter plot with the two types of transactions.
color_map = {0:'red', 1:'blue'}
plt.figure()
for idx, cl in enumerate(np.unique(y)):
   plt.scatter(x = x_test_2d[y==cl,0], y = x_test_2d[y==cl,1], c = color_map[idx], label = cl)
plt.xlabel('X in t-SNE')
plt.ylabel('Y in t-SNE')
plt.legend(loc='upper right')
plt.title('t-SNE visualization of train data')
plt.show()

In [ ]:
df.number_of_payment_faults = 1*(df.number_of_payment_faults > 0)

defaulters = df[df["number_of_payment_faults"] == 1]
non_defaulters = df[df["number_of_payment_faults"] == 0]

defaulters["Defaulter"] = defaulters["car_age_contract_start"]
non_defaulters["Non Defaulter"] = non_defaulters["car_age_contract_start"]
f, ax = plt.subplots(figsize=(12, 6))
ax = sns.kdeplot(defaulters["Defaulter"], shade=True, color="r")
ax = sns.kdeplot(non_defaulters["Non Defaulter"], shade=True, color="b")

In [ ]:
model = mord.LogisticAT()
fitted = model.fit(X, y)
print_coefs(fitted)
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.33, random_state=42)
fitted = model.fit(X_train, y_train)
print(fitted.coef_)
y_predictions = pd.Series(fitted.predict(X_test))
print('y_train.value_counts().sort_index()')
print('y_predictions.value_counts().sort_index()')
print('y_test.value_counts().sort_index()')
cm = confusion_matrix(y_test, y_predictions)
print(cm)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.33, random_state=42)

In [ ]:
model = neighbors.KNeighborsClassifier()
fitted = model.fit(X, y)
knn = neighbors.KNeighborsClassifier()
logistic = linear_model.LogisticRegression(solver='lbfgs', max_iter=1000,                                     multi_class='multinomial')
print('KNN score: %f' % knn.fit(X_train, y_train).score(X_test, y_test))
print('LogisticRegression score: %f'
     % logistic.fit(X_train, y_train).score(X_test, y_test))
predictions_1 = logistic.predict(X_test[0:1000])
pd.Series(predictions_1).value_counts()
print('y_train.value_counts().sort_index()')
print('y_predictions.value_counts().sort_index()')
print('y_test.value_counts().sort_index()')

In [ ]:
from keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from tensorflow.keras import backend
from sklearn.utils import class_weight
from tensorflow.python.keras.models import Sequential

In [ ]:
predictors = X.as_matrix()
predictors = StandardScaler().fit_transform(predictors)
target = to_categorical(data['number_of_payment_faults'])

In [ ]:
model = Sequential()
# The Input Layer :
model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))
# The Hidden Layers :
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(64, kernel_initializer='normal',activation='relu'))
model.add(Dense(32, kernel_initializer='normal',activation='relu'))
model.add(Dense(16, kernel_initializer='normal',activation='relu'))
# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))
# Compile the network :
model.compile(loss='mean_squared_error', optimizer='adam')
# checkpoint
checkpoint = ModelCheckpoint("simple_model.hdf5", monitor='val_loss', verbose=2, save_best_only=True, mode='min')
callbacks_list = [checkpoint] # fit network
history = model.fit(X_train, y_train, epochs=20, batch_size=512,
                        validation_data=(X_test, y_test), verbose=1, shuffle=False,
                        callbacks=callbacks_list)